## 4IZ460 CleverMiner Project

In [2]:
import numpy as np
import pandas as pd
import cleverminer as clm

Cleverminer version  1.0.2  educational version. For production use (bugfixes and keeping compatibility), ask for PRO version.


## Reading datasets and joining them

In [3]:
df = pd.read_csv("./data/csgo_round_snapshots.csv", sep=',')
df.head(5)

,time_left,ct_score,t_score,map,bomb_planted,ct_health,t_health,ct_armor,t_armor,ct_money,...,t_grenade_flashbang,ct_grenade_smokegrenade,t_grenade_smokegrenade,ct_grenade_incendiarygrenade,t_grenade_incendiarygrenade,ct_grenade_molotovgrenade,t_grenade_molotovgrenade,ct_grenade_decoygrenade,t_grenade_decoygrenade,round_winner
0,175.00,0.0,0.0,de_dust2,False,500.0,500.0,0.0,0.0,4000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
1,156.03,0.0,0.0,de_dust2,False,500.0,500.0,400.0,300.0,600.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
2,96.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
3,76.03,0.0,0.0,de_dust2,False,391.0,400.0,294.0,200.0,750.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT
4,174.97,1.0,0.0,de_dust2,False,500.0,500.0,192.0,0.0,18350.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CT


In [4]:
columns = df.columns.tolist()

In [5]:
df_equipment = pd.read_csv("./data/csgo_equipment.csv", sep=',')
df_equipment.head(5)

,name,price
0,Kevlar Vest,$650
1,Kevlar Vest + Helmet,$1000
2,Zeus x27,$200
3,Defuse Kit,$400
4,Molotov,$400


In [6]:
df_weapons = pd.read_csv("./data/csgo_weapons.csv", sep=',')

In [7]:
df_weapons = df_weapons.rename(columns={'Name': 'name', 'Cost': 'price'})
df_weapons.head(5)

,name,Team,price,Kill Award,Kills To Rebuy,Max Speed,RoF,Damage,DPS,Armor Penetration,...,Max Ammo,Reload (CR),Reload (FR),Recoil,Recoil (V),Recoil (H),Spread,Spread Run,Spread Stand,Spread Crouch
0,AK-47,T,$2700,$300,9.00,215.0,600.0,36.0,360.0,77.50%,...,90.0,1.20,2.5,30.0,0.0,70.0,0.6,175.06,6.41,4.81
1,AUG,CT,$3300,$300,11.00,220.0,667.0,28.0,311.0,90.00%,...,90.0,1.50,3.8,26.0,0.0,60.0,0.5,135.45,3.85,2.88
2,AWP,T,$4750,$100,47.50,200.0,41.0,115.0,79.0,97.50%,...,30.0,2.00,3.7,78.0,15.0,20.0,0.2,176.48,80.80,60.60
3,CZ75 Auto,T,$500,$300,1.67,240.0,600.0,33.0,300.0,77.65%,...,12.0,0.00,2.9,27.0,10.0,180.0,3.0,13.41,10.43,7.60
4,Desert Eagle,T,$700,$300,2.33,230.0,267.0,63.0,280.0,93.20%,...,35.0,0.88,2.3,48.2,18.0,60.0,2.0,48.10,4.20,2.18


In [8]:
df_buyable_trans = pd.read_csv("./data/csgo_byuable_to_category_nickname.csv", sep=',')
df_buyable_trans.head(5)

,name,category,nickname
0,AK-47,rifle,ak47
1,AUG,rifle,aug
2,AWP,sniper,awp
3,CZ75 Auto,pistol,cz75auto
4,Desert Eagle,pistol,deagle


In [10]:
# put weapons and equipment together
df_buyables = pd.concat([df_equipment, df_weapons], ignore_index=True)
# join buyables with categories
df_buyables = df_buyables.merge(df_buyable_trans, on='name')
# keep only the columns we plan to use
df_buyables = df_buyables.loc[:, ['name', 'price', 'category', 'nickname']]
# clean price values
df_buyables['price'] = df_buyables['price'].str.replace('$', '', regex=False).astype(int)

In [11]:
def df_to_dict(df_to_convert: pd.DataFrame, key: str, value: str) -> dict:
    """
    This function takes a dataframe, and creates a dictionary, where the keys will be values from key, and value will
    be either a single value (if value parameter is string), or a dictionary, where the keys will be values from value
    (which is a list in this scenario), and the dictionary values will be values in columns in list.
    Parameters
    ----------
    df_to_convert: pd.DataFrame
    key: str
    value: str | list
    Returns
    -------
    dict
    """
    return {x[key]:x[value] if isinstance (value, str) else {xkey: x[xkey] for xkey in value} for x in df_to_convert.to_dict(orient='records')}

In [13]:
buyables_trans = df_to_dict(df_buyables, key='nickname', value=['price', 'category'])
buyables_trans

{'kevlar': {'price': 650, 'category': 'armor'},
 'kevlarhelmet': {'price': 1000, 'category': 'armor'},
 'zeus': {'price': 200, 'category': 'CT equipment'},
 'defuse': {'price': 400, 'category': 'CT equipment'},
 'molotovgrenade': {'price': 400, 'category': 'grenade'},
 'incendiarygrenade': {'price': 600, 'category': 'grenade'},
 'decoygrenade': {'price': 50, 'category': 'grenade'},
 'flashbang': {'price': 200, 'category': 'grenade'},
 'hegrenade': {'price': 300, 'category': 'grenade'},
 'smokegrenade': {'price': 300, 'category': 'grenade'},
 'ak47': {'price': 2700, 'category': 'rifle'},
 'aug': {'price': 3300, 'category': 'rifle'},
 'awp': {'price': 4750, 'category': 'sniper'},
 'cz75auto': {'price': 500, 'category': 'pistol'},
 'deagle': {'price': 700, 'category': 'pistol'},
 'elite': {'price': 500, 'category': 'pistol'},
 'famas': {'price': 2250, 'category': 'rifle'},
 'fiveseven': {'price': 500, 'category': 'pistol'},
 'g3sg1': {'price': 5000, 'category': 'sniper'},
 'galilar': {'pr

In [13]:
def parse_snapshot_column_to_buyable(row: pd.Series, trans: dict) -> tuple[str, str]:
    
    # count t and ct weapons
    # count t and ct categories
    # count t and ct equipment / weapons
    # count overall investment
    
    return buyable_type, buyable_category, buyable_proper_name
    

In [15]:
print(df.columns)

Index(['time_left', 'ct_score', 't_score', 'map', 'bomb_planted', 'ct_health',
       't_health', 'ct_armor', 't_armor', 'ct_money', 't_money', 'ct_helmets',
       't_helmets', 'ct_defuse_kits', 'ct_players_alive', 't_players_alive',
       'ct_weapon_ak47', 't_weapon_ak47', 'ct_weapon_aug', 't_weapon_aug',
       'ct_weapon_awp', 't_weapon_awp', 'ct_weapon_bizon', 't_weapon_bizon',
       'ct_weapon_cz75auto', 't_weapon_cz75auto', 'ct_weapon_elite',
       't_weapon_elite', 'ct_weapon_famas', 't_weapon_famas',
       'ct_weapon_g3sg1', 't_weapon_g3sg1', 'ct_weapon_galilar',
       't_weapon_galilar', 'ct_weapon_glock', 't_weapon_glock',
       'ct_weapon_m249', 't_weapon_m249', 'ct_weapon_m4a1s', 't_weapon_m4a1s',
       'ct_weapon_m4a4', 't_weapon_m4a4', 'ct_weapon_mac10', 't_weapon_mac10',
       'ct_weapon_mag7', 't_weapon_mag7', 'ct_weapon_mp5sd', 't_weapon_mp5sd',
       'ct_weapon_mp7', 't_weapon_mp7', 'ct_weapon_mp9', 't_weapon_mp9',
       'ct_weapon_negev', 't_weapon_negev',